In [29]:
rooms = {
    'main room': { # room 1. Player starts here. 
        'description': "~You wake up on dusty old couch. The room is dimly lit by a candelabra sitting on top of a grand piano. The floor creaks and cobwebs hang from the walls.\n" 
        "And though you can't see anyone else, you're sure you can hear a faint piano melody lingering in the air.~", # Player wakes up on couch and sees room with piano
        'connections': ['bedroom'], # Player must access Bedroom 1 to continue
        'items': ["couch", "grand piano"] 
    },
    'bedroom': { # room 2, connects with main room and living room
        'description': "~The bedroom's wallpaper is yellow and peeling. Silk drapes a large, oak bedframe in the middle of the room.\n"
        "The kingsized bed is covered in silk sheets and many satin pillows, messy from sleep. Someone was just here. You hear the wind howling outside.\n"
        "You need to find a way out of here.~", # player must search bed to find master key for exit
        'connections': ["main room", 'dining room'], 
        'items': ["bed", 'silver key']
    },
    'dining room': { # room 3, player can search dining table and/or use key to exit house
        'description': "You're in a cavernous dining room. Darkness envolpes you. The vast chamber echos with sounds unknown.\n" 
        "Your only source is the glow of a long forgotten fire in the room's massive hearth.\n" 
        "You can see a door at the end of the room. You hope it leads outside.",
        'connections': ["bedroom", 'outside'],
        'items': ["dining table", "fireplace"]
    },
    "outside": { # final challenge. game ends when player exits house
        "description": "You are outside. Fresh air at last!",
        "connections": ["dining room"],
    },
    # Additional rooms...
}


In [26]:
class Game:
    def __init__(self, room, item):
        self.location = room
        self.item = item
        self.room_examined = False
        self.has_key = False
        self.inventory = []  # New line
        
    def select_item(self, new_item):
        # this function sets an atribute to a new value
        self.item = new_item

    def examine_room(self):
        self.room_examined = True
        room_dict = rooms[self.location]
        return room_dict
    

    def move_rooms(self, new_location):
        self.location = new_location
        self.room_examined = False
        self.item = None
    
    def print_room(self):
        room_dict = rooms[self.location]
        return room_dict["description"]

    def take_key(self):
        self.has_key = True
    
    def take_item(self, item):  # New method
        self.inventory.append(item)
        print(f"You took the {item}")
    

In [ ]:
class Door:
    def __init__(self, is_unlocked: bool, connections: set, opens_with: str):
        self.is_unlocked = is_unlocked 
        self.connections = connections
        self.opens_with = opens_with
        

    def unlock(self, random_item: str): 
        if random_item == self.opens_with:
            self.is_unlocked = True 

    def __repr__(self):
        return (f"Door unlocked: {self.is_unlocked}. Connects with: {self.connections}") 
    
    def is_door_unlocked(current_room, target_room, doors):
        for door in doors[current_room]:
            if target_room in door.connections:
                return door.is_unlocked
        return False


In [27]:
player = Game("main room", "couch")

In [40]:
doors_dict = {}
for room in rooms: # loops through each room in rooms dictionary
    doors_dict[room] = [] # creates empty list of rooms in doors dictionary
    print("I am in the room:", room)
    for connection in rooms[room]["connections"]: # loops through all connections in rooms
        door_exists = False # creating a flag to set the state of doors as not exiting
        if connection in doors_dict: # if connection (aka, one of the connecting rooms)
               for existing_door in doors_dict[connection] : #loop through all its existing connections:
                    if {room, connection} == existing_door.connections: # room = bedroom; existing door = main room; connections = {main room, bedroom}, if it exists --> 
                        doors_dict[room].append(existing_door) # we add all the above info (existing_door) to the doors_dict
                        door_exists = True # and mark the door as existing
                        break # break and loop again
        if not door_exists: # if door doesn't exist -->
            print(f"the {room} connects with: {connection}")
            door1 = Door(False, {room, connection}, opens_with="") # --> create a new door
            doors_dict[room].append(door1) # add to door dictionary
print(doors_dict)

I am in the room: main room
the main room connects with: bedroom
I am in the room: bedroom
the bedroom connects with: dining room
I am in the room: dining room
the dining room connects with: outside
I am in the room: outside
{'main room': [Door unlocked: False. Connects with: {'main room', 'bedroom'}], 'bedroom': [Door unlocked: False. Connects with: {'main room', 'bedroom'}, Door unlocked: False. Connects with: {'bedroom', 'dining room'}], 'dining room': [Door unlocked: False. Connects with: {'bedroom', 'dining room'}, Door unlocked: False. Connects with: {'outside', 'dining room'}], 'outside': [Door unlocked: False. Connects with: {'outside', 'dining room'}]}


In [41]:
doors_dict

{'main room': [Door unlocked: False. Connects with: {'main room', 'bedroom'}],
 'bedroom': [Door unlocked: False. Connects with: {'main room', 'bedroom'},
  Door unlocked: False. Connects with: {'bedroom', 'dining room'}],
 'dining room': [Door unlocked: False. Connects with: {'bedroom', 'dining room'},
  Door unlocked: False. Connects with: {'outside', 'dining room'}],
 'outside': [Door unlocked: False. Connects with: {'outside', 'dining room'}]}

In [43]:
doors_dict["main room"][0].is_unlocked=True

doors_dict

{'main room': [Door unlocked: True. Connects with: {'main room', 'bedroom'}],
 'bedroom': [Door unlocked: True. Connects with: {'main room', 'bedroom'},
  Door unlocked: False. Connects with: {'bedroom', 'dining room'}],
 'dining room': [Door unlocked: False. Connects with: {'bedroom', 'dining room'},
  Door unlocked: False. Connects with: {'outside', 'dining room'}],
 'outside': [Door unlocked: False. Connects with: {'outside', 'dining room'}]}

In [31]:
# Game loop
first_time = 1
game_running = True
while game_running:
    if first_time: 
        print(f"{rooms[player.location]['description']}.\n", flush = True)
        first_time = 0
    
    # Show available connections
    current_room = rooms[player.location]
    connections = current_room['connections']

    if player.room_examined == True:
        print(f'''What do you want to do?
            Search for items to help you escape: press 1 
            View connecting rooms: press 2  
            Move to another room: press 3
            Inspect an item: press 4
            Type "exit" to exit game  
            ''', flush = True)
    else:
        print(f'''What do you want to do?
            Search for items to help you escape: press 1 
            View connecting rooms: press 2  
            Move to another room: press 3
            Type "exit" to exit game  
            ''', flush = True)   
    
 
    user_input = input("What do you want to do? ")
    
    if user_input == "1":
        room_info = player.examine_room()
        print("You examined the room and found these items:")
        item_string_and = " & ".join(rooms[player.location]["items"])
        print(item_string_and)
    elif user_input == "2":
       if len(connections) > 0: # turn into a function, 
            print("Rooms you can move to:")
            for i, room in enumerate(connections):
                print(f"    - {room}")
    elif user_input == "3":
         if len(connections) > 0:
            connections_string = " and ".join(connections)
            print(f"What room do you want to open?\nType '{connections_string}' to move there.")

            new_room = input(f"What room do you want to open? {connections}: ")
            if new_room in connections:
                first_time = 1
                if doors_dict[new_room].is_unlocked:
                    print("It entered")
                # Check if trying to go outside
                if new_room == "outside" and player.has_key:
                    print("You use the Silver Key to unlock the door and escape!")
                    print("Congratulations! You won!")
                    game_running = False
                elif new_room == "outside" and not player.has_key:
                    print("The door is locked. You need a key to escape.")
                else:
                    player.move_rooms(new_room)
                    print(f"You moved to {new_room}") # print(you have moved to {display_string(new_room})
            else:
                print("You can't go there from here.")
         else:
            print("No exits available.")
    elif user_input == "4":
        print(f"Which item do you want to inspect? Type item name to inspect item.")
        user_input = input("Which items do you want to select? Type item name: ")
        if user_input.lower() == "silver key":
            player.take_key()
            print("Congratulations! This key is special. Use it to escape!")
        else: 
            print("Nothing special about this item. Keep searching!")    
    elif user_input.lower() == "exit":
        print("You ended the game. Good-bye!")
        game_running = False
    else:
        print("I don't understand that command.")

~The bedroom's wallpaper is yellow and peeling. Silk drapes a large, oak bedframe in the middle of the room.
The kingsized bed is covered in silk sheets and many satin pillows, messy from sleep. Someone was just here. You hear the wind howling outside.
You need to find a way out of here.~.

What do you want to do?
            Search for items to help you escape: press 1 
            View connecting rooms: press 2  
            Move to another room: press 3
            Type "exit" to exit game  
            
What room do you want to open?
Type 'main room and dining room' to move there.


AttributeError: 'list' object has no attribute 'is_unlocked'

In [ ]:
elif user_input == "3":
         if len(connections) > 0:
            connections_string = " and ".join(connections)
            print(f"What room do you want to open?\nType '{connections_string}' to move there.")
            new_room = input(f"What room do you want to open? {connections}: ")
            if new_room in connections:
                first_time = 1
                if doors_dict[new_room].is_unlocked:
                    print("It entered")
                    # Then you can use your previous code
                # else
                    print("The door is locked. You need a key to escape.")
                # Check if trying to go outside
                if new_room == "outside" and player.has_key:
                    print("You use the Silver Key to unlock the door and escape!")
                    print("Congratulations! You won!")
                    game_running = False
                elif new_room == "outside" and not player.has_key:
                    print("The door is locked. You need a key to escape.")
                else:
                    player.move_rooms(new_room)
                    print(f"You moved to {new_room}") # print(you have moved to {display_string(new_room})
            else:
                print("You can't go there from here.")
         else:
            print("No exits available.")
